# 추천사유 생성

## 결과물 형태

In [5]:
# Schema
from typing_extensions import Annotated
from pydantic import Field, BaseModel

class IDRelevance(BaseModel):
    relevant_id: Annotated[
        list[str],
        Field(
            ..., 
            description=(
                "데이터의 ID 목록"
            ), 
        )
    ]
    reason: Annotated[
        list[str],
        Field(
            ..., 
            description="각 ID가 선정된 이유를 설명하는 문자열 목록. relevant_id와 인덱스가 일치해야 합니다.",
        )
    ]

## 프롬프트

In [6]:
# Prompt
from langchain_core.prompts import PromptTemplate

reason_template = '''
당신은 데이터 과학자입니다. 아래는 연구 데이터 목록입니다.

각 데이터 혹은 논문 항목은 다음 컬럼을 가지고 있습니다:
- ID: 각 데이터의 고유키
- 제목
- 설명
- 키워드

[목표]
모든 항목에 대해 해당 항목의 선정 사유를 작성해 주세요.

**중요: relevant_id와 reason의 개수는 반드시 동일해야 합니다.**

[Input]
연구 주제: {title}
연구 설명: {description}
키워드: {keyword}

[Data]
데이터 목록:
{data}

[Output]
다음 형식의 JSON을 출력하세요. relevant_id와 reason의 길이가 정확히 일치해야 합니다:

{{
  "relevant_id": ["ID1", "ID2", "ID3"],
  "reason": ["이유1", "이유2", "이유3"]
}}
'''

reason_prompt = PromptTemplate.from_template(reason_template)


## 작동 방식

In [7]:
# 예시 데이터
import json
import pandas as pd

# title, description, keyword
with open("../data/input_data.json", "r", encoding="utf-8") as f:
    input_data = json.load(f)

try:
    title = input_data['dataset_title_etc_main']
    description = input_data['dataset_expl_etc_main']
    keyword = input_data['dataset_expl_etc_main']
    input_id = input_data['svc_id']

except:
    items = input_data["MetaData"]["recordList"]["record"]["item"]
    title = next(i["#text"] for i in items if i["@metaCode"] == "Title")
    description = next(i["#text"] for i in items if i["@metaCode"] == "Abstract")
    keyword = next(i["#text"] for i in items if i["@metaCode"] == "Keyword")
    input_id = next(i["#text"] for i in items if i["@metaCode"] == "CN")
    
# data
df_article = pd.read_csv('../data/search_results_article.csv', encoding='UTF-8', low_memory=False)
df_data = pd.read_csv('../data/search_results_dataset.csv', encoding='UTF-8', low_memory=False)

cleaned_df_data = (
    df_data[
        ['svc_id', 'dataset_title_etc_main', 'dataset_expl_etc_main','dataset_pub_dt_pc', 'dataset_kywd_etc_main', 'dataset_creator_etc_main', 'dataset_lndgpg', 'query']
    ]
    .rename(
        columns={
            'svc_id': 'ID',
            'dataset_title_etc_main': 'title',
            'dataset_expl_etc_main': 'description',
            'dataset_pub_dt_pc': 'pubyear',
            'dataset_kywd_etc_main': 'keyword',
            'dataset_creator_etc_main': 'author',
            'dataset_lndgpg': 'URL',
        }
    )
)
cleaned_df_data['category'] = 'dataset'

cleaned_df_arti = (
    df_article[
        ['CN', 'Title', 'Abstract', 'Pubyear', 'Keyword', 'Author', 'ContentURL', 'query']
    ]
    .rename(
        columns={
            'CN': 'ID',
            'Title': 'title',
            'Abstract': 'description',
            'Pubyear': 'pubyear',
            'Keyword': 'keyword',
            'Author': 'author',
            'ContentURL': 'URL'
        }
    )
)
cleaned_df_arti['category'] = 'article'

df = pd.concat([cleaned_df_arti, cleaned_df_data], ignore_index=True)

# relevance_data
relevance_df = pd.read_csv('../data/relevance_results.csv', encoding='UTF-8', low_memory=False)

In [9]:
# Node
from langchain_openai import ChatOpenAI

relevant_ids = relevance_df['ID'].tolist()
filtered_df = df[df['ID'].isin(relevant_ids)]

prompt = reason_prompt.invoke(
    {
        'title': title, 
        'description': description,
        'keyword': keyword,
        'data': filtered_df[['ID', 'title', 'description', 'keyword']].to_dict(orient="records"),
    }
)

sllm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

structured_sllm = sllm.with_structured_output(IDRelevance)
res = structured_sllm.invoke(prompt)
    
tmp = pd.DataFrame({
    'ID': res.relevant_id,
    'reason': res.reason
})

relevance_df = pd.merge(
    relevance_df[['ID', 'relevance']],
    tmp,
    on='ID',
    how='left'
)

relevance_df.to_csv('../data/relevance_results.csv', index=False, encoding='utf-8')

display(relevance_df)

,ID,relevance,reason
0,d023e479d6a3e09f0d7988cf38a4436b,92.56636,"이 데이터는 O.Granite Harbor에서 수집된 중력 코어 샘플로, 연구 주제..."
1,ff96e62579ae3046d133440562968c39,92.31045,"이 데이터는 O.Granite Harbor 근처에서 수집된 중력 코어 샘플로, 연구..."
2,21f628ecb675030dedda1149f466adae,91.94792,"이 데이터는 Ross Sea에서의 중력 코어 샘플링을 다루고 있으며, 연구 주제와 ..."
3,e83e64b3b4ea6a9982da08310ea27b1b,91.60432,"이 데이터는 W.Beanfort Island에서 수집된 중력 코어 샘플로, 연구 주..."
4,83d26621eaf49e20d987f3d7d4005122,91.57433,"이 데이터는 Ross Sea에서의 중력 코어 샘플링을 다루고 있으며, 연구 주제와 ..."
